과제 1

In [3]:
## 1번 모델 : facebook/detr-resnet-50

from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image, ImageDraw

# 로컬 이미지 파일 경로로 수정
image_path = "/object_detection.jpg"
image = Image.open(image_path)

# 모델과 프로세서 로드
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

# 이미지 전처리
inputs = processor(images=image, return_tensors="pt")

# 모델 실행
outputs = model(**inputs)

# 결과를 COCO API 형식으로 변환
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

# 결과 이미지 생성 및 저장
result_image = image.copy()
draw = ImageDraw.Draw(result_image)

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    label_str = model.config.id2label[label.item()]
    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )

    # 박스 그리기
    draw.rectangle(box, outline="red", width=3)

    # 라벨과 신뢰도 표시
    draw.text((box[0], box[1]), f"Detected {label_str} with confidence {round(score.item(), 3)} at location {box}", fill="red")

# 결과 이미지 저장
result_image.save("/result_image.jpg")

# 결과 출력
result_image.show()


Detected kite with confidence 0.988 at location [318.13, 93.78, 393.97, 156.52]
Detected person with confidence 0.999 at location [597.72, 206.31, 669.59, 415.73]
Detected kite with confidence 0.997 at location [617.86, 379.51, 731.87, 433.64]
Detected person with confidence 0.999 at location [338.05, 232.54, 406.47, 430.02]
Detected kite with confidence 0.958 at location [573.83, 111.01, 593.08, 165.8]
Detected person with confidence 0.998 at location [420.16, 268.72, 479.21, 425.0]
Detected person with confidence 0.98 at location [508.74, 250.86, 567.85, 426.96]


In [5]:
## 2번 모델 : YOLOS (tiny-sized) model
from transformers import YolosImageProcessor, YolosForObjectDetection
from PIL import Image, ImageDraw
import torch

# 로컬 이미지 파일 경로로 수정
image_path = "/object_detection.jpg"
image = Image.open(image_path)

# 모델과 프로세서 로드
model = YolosForObjectDetection.from_pretrained('hustvl/yolos-tiny')
image_processor = YolosImageProcessor.from_pretrained("hustvl/yolos-tiny")

# 이미지 전처리
inputs = image_processor(images=image, return_tensors="pt")

# 모델 실행
outputs = model(**inputs)

# 결과를 COCO API 형식으로 변환
target_sizes = torch.tensor([image.size[::-1]])
results = image_processor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[0]

# 결과 출력
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
        f"Detected {model.config.id2label[label.item()]} with confidence "
        f"{round(score.item(), 3)} at location {box}"
    )

# 결과 이미지 생성 및 저장
result_image = image.copy()
draw = ImageDraw.Draw(result_image)

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    label_str = model.config.id2label[label.item()]

    # 박스 그리기
    draw.rectangle(box, outline="blue", width=3)

    # 라벨과 신뢰도 표시
    draw.text((box[0], box[1]), f"Detected {label_str} with confidence {round(score.item(), 3)} at location {box}", fill="blue")

# 결과 이미지 저장
result_image.save("/result_image_yolos.jpg")

# 결과 출력
result_image.show()

Detected person with confidence 0.998 at location [512.8, 251.33, 573.15, 430.01]
Detected person with confidence 0.998 at location [413.5, 269.65, 480.1, 419.68]
Detected person with confidence 1.0 at location [336.14, 233.13, 406.06, 429.8]
Detected kite with confidence 0.938 at location [572.64, 114.6, 589.06, 160.53]
Detected kite with confidence 0.902 at location [614.21, 375.2, 724.22, 437.49]
Detected person with confidence 0.999 at location [591.63, 207.01, 674.55, 415.98]


# **성능 비교 **

첫 번째 모델이 두 번째 모델에 비해 성능이 뛰어나다. 두 번째 모델은 연을 2개 감지한 데에 비해 첫 번째 모델은 연 3개를 정확하게 감지하고 있다.

과제 2